In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
from fake_useragent import UserAgent
from datetime import datetime
import time
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [37]:
url = "https://smtv.io/program/listwithgame/default?parent=43&child=all&day_length=7&date=2020-05-30&now=Y&filter_live=Y"
res = requests.get(url,headers=headers)
test = json.loads(res.text)



In [38]:
test["result"]




[{'date': '2020-05-30',
  'data': [{'solarApi': {'hTotalScore': None,
     'aTotalScore': None,
     'time_info': 'LIVE',
     'situation': ['半场 0-0', '角球 0-0'],
     'extradata': {'1': [None, None],
      '2': [None, None],
      '3': [None, None],
      '4': [None, None],
      '5': [None, None],
      '6': [None, None],
      '7': [None, None],
      '8': [None, None],
      '13': [None, None],
      '15': [None, None],
      '16': [None, None],
      '17': [None, None],
      '19': [None, None],
      '21': [None, None],
      '22': [None, None],
      '23': [None, None],
      '24': [None, None],
      '25': [None, None],
      'fats': [None, None],
      'fpks': [None, None],
      'round': None,
      'home_position': None,
      'away_position': None},
     'hid': '319608f9014b11ea92ed42010a8c0055',
     'aid': None,
     'hTeamMatchesUrl': 'https://static04.dsoog.cn/teams/9014b11ea92ed42010a8c0055/319608f9014b11ea92ed42010a8c0055.json',
     'aTeamMatchesUrl': ''},
    'id': 5

In [ ]:
# smtv 體育
# https://smtv.io/program/43
# smtv 娛樂
# https://smtv.io/program/44
# smtv 電競
# https://smtv.io/program/81

#------------------------------

# smzb 體育
# https://smzb.io/program/1
# smzb 娛樂
# https://smzb.io/program/2
# smzb 電競    
# https://smzb.io/program/45

#------------------------------

# ggzb 電競
# https://ggzb.io/

#------------------------------

# lj88 體育
# https://lj88.tv/

#------------------------------

# hqzb 體育
# https://hqzb.tv/program

#------------------------------